Importing Libraries

In [11]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import feature

Implemention of LBP

In [12]:
class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		self.numPoints = numPoints
		self.radius = radius
	def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = feature.local_binary_pattern(image, self.numPoints,self.radius, method="uniform")
		
		(hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, self.numPoints + 3),range=(0, self.numPoints + 2))
		
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)

		return hist

Feature Extraction

In [13]:
%%capture
# x is feature of images, including HOG and LBP
x = []
# Make an instance of lbp class and set neighbour points and radius
lbp_c = LocalBinaryPatterns(24, 2) # number of points, radius

# start reading images one by one and do feature extraction
for i in range(1,4001):
    num = str(i).zfill(4)
    img = cv2.imread('newimg/img'+ num +'.jpg')
    img = cv2.resize(img, (100,100))

    # required converts for HOG and LBP
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    hog_features = hog(img, orientations=30, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualize=False, channel_axis=-1) # HOG
    lbp_features = lbp_c.describe(gray) # LBP

    # Concatination of HOG and LBP and append into x array
    x.append(np.concatenate((hog_features,lbp_features),axis = 0))

# Read the label, the first column of the file
y = pd.read_csv('labels.txt',' ',header=None)
y = y.iloc[:,0].values

Starting the ML model, using SVC
* importing SVC class
* importing metrics for evaluation
* train_test_split for seperate train and test

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 20)

Learn the SVC model
Then Fit

In [15]:
classifier = SVC(kernel = 'linear', random_state = 13)
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

Evaluations

Good accuracy with linear kernel

In [16]:
print('Score = ',classifier.score)
print('Confusion Matrix = ')
print(confusion_matrix(y_test, y_pred))
print('Accuracy = ', accuracy_score(y_test, y_pred))
print('F1 = ', f1_score(y_test, y_pred))
print('Precision = ', precision_score(y_test, y_pred))
print('Recall = ', recall_score(y_test, y_pred))

Score =  <bound method ClassifierMixin.score of SVC(kernel='linear', random_state=13)>
Confusion Matrix = 
[[368  79]
 [ 98 455]]
Accuracy =  0.823
F1 =  0.8371665133394665
Precision =  0.8520599250936329
Recall =  0.8227848101265823


Saving the built model

In [17]:
import pickle

filename = 'finalized_smile_detection_SVC_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

# also loading 
# loaded_model = pickle.load(open(filename, 'rb'))

Input your image
* Read a image
* Detect its face
* Shows the detected face 
* Do feature extraction
* Predict if it is smile or not

In [18]:
# my image is in the folder with the name of photo.jpg
inn = input()
img = cv2.imread(inn)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(gray, 1.2, 6)

detected = 0
if len(faces) != 1:
    j = 1
    while(len(faces) != 1):
        faces = face_cascade.detectMultiScale(gray, 1.1, j)
        j+=1
        if(len(faces) == 0):
            break
    if len(faces) != 1:
        detected = 0
    else:
        (x, y, w, h) = faces[0]
        res = img[y:y+h,x: x+w]
        detected = 1
else:
    (x, y, w, h) = faces[0]
    res = img[y:y+h,x: x+w]
    detected = 1

cv2.imshow('image', res)
cv2.waitKey(0)

print(detected)
if detected:
    res = cv2.resize(res, (100,100))
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    img = cv2.cvtColor(res,cv2.COLOR_BGR2RGB)

    hog_features = hog(img, orientations=30, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualize=False, channel_axis=-1) # HOG
    lbp_features = lbp_c.describe(gray)

    if classifier.predict([np.concatenate((hog_features,lbp_features),axis = 0)]):
        print("Smiling! :) ")
    else:
        print("Not smiling! :(")

1
Smiling! :) 
